In [1]:
from test import test_models
# import yaml

# with open('test_config.yaml', 'r') as file:
#     test_config = yaml.safe_load(file)


In [2]:
models = [
    # {
    #     'model': 
    #         {
    #             'name': 'UnetPlusPlus',
    #             'downsample_target': False,
    #             'load_args': 
    #                 {
    #                     'pre_trained_name': 'imagenet', 
    #                     'encoder_name': 'resnet34'
    #                 }
    #         },
    #     'checkpoint': './expirements/unet_plus_plus_baseline/checkpoint-1000',
    # },
    # {
    #     'model': 
    #         {
    #             'name': 'UnetPlusPlus',
    #             'downsample_target': False,
    #             'load_args': 
    #                 {
    #                     'pre_trained_name': 'imagenet', 
    #                     'encoder_name': 'resnet101',
    #                 }
    #         },
    #     'checkpoint': './expirements/unet_plus_plus_resnet101/checkpoint-1000',
    # },
    {
        'model': 
            {
                'name': 'SegformerForSemanticSegmentation',
                'downsample_target': False,
                'load_args': 
                {
                    'pre_trained_name': 'nvidia/mit-b0'
                }
            },
        'checkpoint': './expirements/segformer_mit-b0/checkpoint-1000',
    },       
]

In [3]:
metrics = test_models(models=models, resource='./test_config.yaml')

Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing WrappedSegformerForSemanticSegmentation: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing WrappedSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WrappedSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WrappedSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.classifier.bi

WrappedSegformerForSemanticSegmentation: <All keys matched successfully>


100%|██████████| 196/196 [00:00<00:00, 8880.77it/s]


Warning! dataset/sessile-main-Kvasir-SEG/kavsir_bboxes.json file not found, dataset will be without bboxes


getting predictions: 100%|██████████| 196/196 [00:19<00:00, 10.01it/s]


torch.Size([1, 2, 128, 128]) torch.Size([1, 512, 512])


AttributeError: 'builtin_function_or_method' object has no attribute 'shape'

In [ ]:
metrics

[{'model_name': 'WrappedUnetPlusPlus',
  'params': 26078754,
  'metrics': {'dice_group_0': 0.9243648,
   'iou_group_0': 0.8625277,
   'dice_group_1': 0.9665257,
   'iou_group_1': 0.9353107,
   'dice_group_2': 0.96991986,
   'iou_group_2': 0.9418973,
   'dice_group_3': 0.96910036,
   'iou_group_3': 0.94123316,
   'dice_group_4': 0.97895277,
   'iou_group_4': 0.9587733,
   'dice': 0.93328035,
   'iou': 0.8780151}},
 {'model_name': 'WrappedUnetPlusPlus',
  'params': 26078754,
  'metrics': {'dice_group_0': 0.9243648,
   'iou_group_0': 0.8625277,
   'dice_group_1': 0.9665257,
   'iou_group_1': 0.9353107,
   'dice_group_2': 0.96991986,
   'iou_group_2': 0.9418973,
   'dice_group_3': 0.96910036,
   'iou_group_3': 0.94123316,
   'dice_group_4': 0.97895277,
   'iou_group_4': 0.9587733,
   'dice': 0.93328035,
   'iou': 0.8780151}},
 {'model_name': 'WrappedUnetPlusPlus',
  'params': 26078754,
  'metrics': {'dice_group_0': 0.9243648,
   'iou_group_0': 0.8625277,
   'dice_group_1': 0.9665257,
   'i

In [8]:
from tabulate import tabulate

headers = ['model'] + list(metrics[0]['metrics'].keys())
table = [[metrics[i]['model_name'] + f' ({metrics[i]["params"]/1e+6:.3}M)'] + [f"{metrics[i]['metrics'][k]:.3}" for k in headers[1:]] for i in range(len(metrics))]
print(tabulate(table, headers=headers))

model                                      dice_group_0                               iou_group_0                                dice_group_1                               iou_group_1                                 dice_group_2                               iou_group_2                                 dice_group_3                                iou_group_3                                 dice_group_4                               iou_group_4                                 dice
-----------------------------------------  -----------------------------------------  -----------------------------------------  -----------------------------------------  ------------------------------------------  -----------------------------------------  ------------------------------------------  ------------------------------------------  ------------------------------------------  -----------------------------------------  ------------------------------------------  ---------------------------------------

In [ ]:
test_config

In [ ]:
metrics, pred, labels = test_model('./test_config.yaml', return_pred=False)

In [ ]:
from tabulate import tabulate

table = [[k, f'{v:.3}'] for k, v in metrics.items()]
print(tabulate(table))

In [ ]:
from PIL import Image
from torchvision.transforms import ToPILImage
import torch.nn.functional as F
to_pill_image = ToPILImage()

In [ ]:
from src.data_utils.dataset import load_dataset

test_dataset = load_dataset(r'./test_config.yaml')

In [ ]:
idx = 42

logits = pred[idx]
target_mask = labels[idx]

logits = F.softmax(logits, dim=0)
pred_mask = logits.argmax(dim=0)

pred_mask_tensor = pred_mask.bool()

pred_mask = to_pill_image(pred_mask.float())
target_mask = to_pill_image(target_mask.float())

# pred_mask.show()
# target_mask.show()

In [ ]:
from src.data_utils.utils import plot_mask_over_image

data_ = test_dataset.__getitem__(idx)
input_ = data_['input']
target_ = data_['target'].bool()

plot_mask_over_image(input_, target_)
plot_mask_over_image(input_, pred_mask_tensor)

In [ ]:
from train import train_setup

In [ ]:
trainer = train_setup(r'./train_config.yaml')

In [ ]:
# set(trainer.train_dataset.data_names).intersection(set(trainer.eval_dataset.data_names))

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
print(trainer.train_dataset.dataset[0]['input'].shape)
trainer.train_dataset.dataset[0]['target'].shape

In [ ]:
print(trainer.eval_dataset[0]['input'].shape)
trainer.eval_dataset[0]['target'].shape

In [ ]:
trainer.train_dataset.dataset.augmentator()